# 1. 层和块

In [1]:
import tensorflow as tf

In [3]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(10),
])
X = tf.random.uniform((2,20))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.39291954,  0.2525106 ,  0.02978083,  0.13245568, -0.3161534 ,
         0.25431594,  0.07534377, -0.06229079, -0.05107045, -0.3263515 ],
       [-0.3283259 ,  0.14908814, -0.18172464,  0.09874249, -0.26747587,
         0.09416143, -0.04878247,  0.10114376, -0.05776639, -0.43654343]],
      dtype=float32)>

## 1.1 自定义块

In [4]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.hidden = tf.keras.layers.Dense(units=256,activation='relu')
        self.out = tf.keras.layers.Dense(units=10)
    def call(self,X):
        return self.out(self.hidden((X)))

In [5]:
net = MLP()

In [6]:
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.26748484,  0.07539991,  0.21809646,  0.08414759,  0.02512698,
         0.3535815 ,  0.26445916,  0.16719699, -0.02249558, -0.1051607 ],
       [-0.21155119,  0.23459646,  0.20549837,  0.16302368, -0.01029655,
         0.10908729,  0.19798984,  0.09204209,  0.13118033,  0.06843205]],
      dtype=float32)>

## 1.2 Sequential

为了构建Sequential，我们只需要定义两个关键函数：
1. 一种将块逐个追加到列表中的函数
2. 一种前向传播函数，用于将输入按追加块的顺序传递给块组成的链条

In [9]:
class MySequential(tf.keras.Model):
    def __init__(self,*args):
        super().__init__()
        self.modules = []
        for block in args:
            self.modules.append(block)
            
    def call(self,X):
        for module in self.modules:
            X = module(X)
        return X

In [10]:
net = MySequential(
    tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.22040497,  0.09712207, -0.49343053, -0.26114163, -0.09403025,
         0.18024832, -0.2003344 , -0.04766444, -0.09108949,  0.16741958],
       [-0.0022357 , -0.00905097, -0.41821143, -0.16958472,  0.13493344,
         0.07799095, -0.12572452, -0.04986995, -0.10445809, -0.06300616]],
      dtype=float32)>

## 1.3 在前向传播函数中执行代码

In [7]:
class FixedHiddenMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.rand_weight = tf.constant(tf.random.uniform((20,20)))
        self.dense = tf.keras.layers.Dense(20,activation='relu')
        
    def call(self,inputs):
        X = self.flatten(inputs)
        X = tf.nn.relu(tf.matmul(X,self.rand_weight)+1)
        X = self.Dense(X)
        while tf.reduce_sum(tf.math.abs(X)) > 1:
            X /= 2
        return tf.reduce_sum(X)

# 2. 参数管理

In [1]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation=tf.nn.relu),
    tf.keras.layers.Dense(1),
])

X = tf.random.uniform((2, 4))
net(X)

2022-01-07 12:55:55.528021: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-07 12:55:55.592607: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-07 12:55:55.592845: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-07 12:55:55.593765: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.5565221],
       [-0.5166136]], dtype=float32)>

## 2.1 参数访问

当通过Sequential类定义模型时， 我们可以通过索引来访问模型的任意层。

In [9]:
print(net.layers[2].weights)

[<tf.Variable 'dense_5/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[0.06600761],
       [0.4388789 ],
       [0.1059742 ],
       [0.1361829 ]], dtype=float32)>, <tf.Variable 'dense_5/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


In [10]:
print(type(net.layers[2].weights[1]))

<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>


In [11]:
print(net.layers[2].weights[1])

<tf.Variable 'dense_5/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>


In [12]:
print(tf.convert_to_tensor(net.layers[2].weights[1]))

tf.Tensor([0.], shape=(1,), dtype=float32)


In [13]:
print(net.layers[1].weights)

[<tf.Variable 'dense_4/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.10635978, -0.70920706,  0.31924087, -0.34567642],
       [ 0.8032555 ,  0.06991482,  0.654339  ,  0.19643372],
       [-0.09552348, -0.43398106, -0.18385684, -0.71140313],
       [ 0.574472  ,  0.210401  , -0.35696286,  0.50948626]],
      dtype=float32)>, <tf.Variable 'dense_4/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]


In [14]:
print(net.get_weights())

[array([[-0.10635978, -0.70920706,  0.31924087, -0.34567642],
       [ 0.8032555 ,  0.06991482,  0.654339  ,  0.19643372],
       [-0.09552348, -0.43398106, -0.18385684, -0.71140313],
       [ 0.574472  ,  0.210401  , -0.35696286,  0.50948626]],
      dtype=float32), array([0., 0., 0., 0.], dtype=float32), array([[0.06600761],
       [0.4388789 ],
       [0.1059742 ],
       [0.1361829 ]], dtype=float32), array([0.], dtype=float32)]


In [15]:
net.get_weights()[1]

array([0., 0., 0., 0.], dtype=float32)

In [16]:
def block1(name):
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4, activation=tf.nn.relu)],
        name=name)

def block2():
    net = tf.keras.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add(block1(name=f'block-{i}'))
    return net

rgnet = tf.keras.Sequential()
rgnet.add(block2())
rgnet.add(tf.keras.layers.Dense(1))
rgnet(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.29809207],
       [0.44140223]], dtype=float32)>

In [17]:
print(rgnet.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_4 (Sequential)   (2, 4)                    80        
                                                                 
 dense_10 (Dense)            (2, 1)                    5         
                                                                 
Total params: 85
Trainable params: 85
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
rgnet.layers[0].layers[1].layers[1].weights[1]

<tf.Variable 'dense_7/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>

## 2.2 参数初始化

In [21]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,activation='relu',
        kernel_initializer=tf.random_normal_initializer(mean=0,stddev=0.01),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1)])

In [22]:
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.00842498],
       [0.00336687]], dtype=float32)>

In [23]:
net.weights[0]

<tf.Variable 'dense_11/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.0072488 , -0.0149648 ,  0.00616377,  0.00365585],
       [ 0.00654745,  0.00921178, -0.01984715,  0.00516886],
       [ 0.00296929, -0.00865509,  0.00471181,  0.00853929],
       [ 0.01208029, -0.01816077,  0.00402742,  0.01782557]],
      dtype=float32)>

In [24]:
net.weights[1]

<tf.Variable 'dense_11/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>

In [25]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.Constant(1),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1),
])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_13/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32)>,
 <tf.Variable 'dense_13/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [26]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.GlorotUniform()),
    tf.keras.layers.Dense(
        1, kernel_initializer=tf.keras.initializers.Constant(1)),
])

net(X)
print(net.layers[1].weights[0])
print(net.layers[2].weights[0])

<tf.Variable 'dense_15/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 0.11521745,  0.7147353 ,  0.06997406,  0.12121969],
       [ 0.7620409 ,  0.3297978 , -0.34653783,  0.21742517],
       [ 0.05747563, -0.15420163, -0.47560447,  0.56439906],
       [ 0.3862701 , -0.13622272,  0.29210275, -0.3564487 ]],
      dtype=float32)>
<tf.Variable 'dense_16/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[1.],
       [1.],
       [1.],
       [1.]], dtype=float32)>


In [3]:
class MyInit(tf.keras.initializers.Initializer):
    def __call__(self, shape, dtype=None):
        data=tf.random.uniform(shape, -10, 10, dtype=dtype)
        factor=(tf.abs(data) >= 5)
        factor=tf.cast(factor, tf.float32)
        return data * factor

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=MyInit()),
    tf.keras.layers.Dense(1),
])

net(X)
print(net.layers[1].weights[0])

<tf.Variable 'dense_4/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.       , -0.       , -7.5169373,  8.119707 ],
       [-0.       ,  6.338682 , -7.4084473, -0.       ],
       [-0.       , -9.099838 , -0.       ,  0.       ],
       [-7.2869635,  6.2266045,  9.591053 , -6.185658 ]], dtype=float32)>


## 2.3 参数绑定

有时我们希望在多个层间共享参数： 我们可以定义一个稠密层，然后使用它的参数来设置另一个层的参数。

In [2]:
# tf.keras的表现有点不同。它会自动删除重复层
shared = tf.keras.layers.Dense(4, activation=tf.nn.relu)
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    shared,
    shared,
    tf.keras.layers.Dense(1),
])

net(X)
# 检查参数是否不同
print(len(net.layers) == 3)

True


# 3. 延后初始化

In [4]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

In [5]:
[net.layers[i].get_weights() for i in range(len(net.layers))]

[[], []]

# 4. 自定义层

In [6]:
import tensorflow as tf

In [7]:
class CenterdLayer(tf.keras.Model):
    def __init__(self):
        super().__init__()
    def call(self,inputs):
        return inputs-tf.reduce_mean(inputs)

In [9]:
layer = CenterdLayer()
layer(tf.constant([1,2,3,4,5]))

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([-2, -1,  0,  1,  2], dtype=int32)>

In [11]:
net = tf.keras.Sequential([tf.keras.layers.Dense(128), CenterdLayer()])

In [12]:
Y = net(tf.random.uniform((4, 8)))
tf.reduce_mean(Y)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [13]:
class MyDense(tf.keras.Model):
    def __init__(self,units):
        super().__init__()
        self.units = units
        
    def build(self,X_shape):
        self.weight = self.add_weight(name='weight',shape=[X_shape[-1],self.units],
                                      initializer=tf.random_normal_initializer())
        self.bias = self.add_weight(name='bias',shape=[self.units],
                                    initializer=tf.zeros_initializer())
    
    def call(self,X):
        linear = tf.matmul(X,self.weight)+self.bias
        return tf.nn.relu(linear)

In [14]:
dense = MyDense(3)
dense(tf.random.uniform((2, 5)))
dense.get_weights()

[array([[-0.08016359,  0.04054431,  0.04162988],
        [-0.06544625,  0.0113687 ,  0.02372276],
        [ 0.06601661, -0.09021402,  0.0078855 ],
        [-0.05890216, -0.03741097,  0.07294768],
        [-0.04298727,  0.05240272, -0.00429331]], dtype=float32),
 array([0., 0., 0.], dtype=float32)]

# 5. 读写文件

## 5.1 加载和保存张量

In [15]:
import numpy as np
import tensorflow as tf

In [16]:
x = tf.range(4)

In [23]:
#保存单个张量
np.save('x-file.npy', x)

In [24]:
x2 = np.load('x-file.npy', allow_pickle=True)
x2

array([0, 1, 2, 3], dtype=int32)

In [25]:
#保存张量列表
y = tf.zeros(4)
np.save('xy-files.npy', [x, y])
x2, y2 = np.load('xy-files.npy', allow_pickle=True)
(x2, y2)

(array([0., 1., 2., 3.]), array([0., 0., 0., 0.]))

In [26]:
#保存字典
mydict = {'x': x, 'y': y}
np.save('mydict.npy', mydict)
mydict2 = np.load('mydict.npy', allow_pickle=True)
mydict2

array({'x': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([0, 1, 2, 3], dtype=int32)>, 'y': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>},
      dtype=object)

## 5.2 加载和保存模型参数

In [27]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.hidden = tf.keras.layers.Dense(units=256,activation='relu')
        self.out = tf.keras.layers.Dense(units=10)
    def call(self,inputs):
        x = self.flatten(inputs)
        x = self.hidden(x)
        return self.out(x)

In [28]:
net = MLP()
X = tf.random.uniform((2,20))
Y = net(X)

In [29]:
Y

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.12652433, -0.12589364,  0.2272445 ,  0.23120558, -0.15709354,
         0.54463804,  0.06234664, -0.12878226,  0.11383081,  0.20325705],
       [-0.04879332, -0.09252413,  0.08910849,  0.18748908, -0.15910994,
         0.2418121 , -0.01309797, -0.11234334,  0.16654095,  0.13663894]],
      dtype=float32)>

In [30]:
net.save_weights('mlp.params')

In [31]:
clone = MLP()
clone.load_weights('mlp.params')

In [32]:
Y_clone = clone(X)
Y_clone == Y

<tf.Tensor: shape=(2, 10), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])>

# 6. GPU

使用nvidia-smi命令来查看显卡信息

In [33]:
!nvidia-smi

Fri Jan  7 14:22:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:2D:00.0  On |                  N/A |
|  0%   43C    P8    13W / 240W |   6765MiB /  7959MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

我们可以指定用于存储和计算的设备。

In [34]:
import tensorflow as tf

tf.device('/CPU:0'), tf.device('/GPU:0'), tf.device('/GPU:1')

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7f732c9ae040>,
 <tensorflow.python.eager.context._EagerDeviceContext at 0x7f732c9aea80>)

查询可用的GPU数量

In [35]:
len(tf.config.experimental.list_physical_devices('GPU'))

1

In [36]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if len(tf.config.experimental.list_physical_devices('GPU')) >= i + 1:
        return tf.device(f'/GPU:{i}')
    return tf.device('/CPU:0')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    num_gpus = len(tf.config.experimental.list_physical_devices('GPU'))
    devices = [tf.device(f'/GPU:{i}') for i in range(num_gpus)]
    return devices if devices else [tf.device('/CPU:0')]

我们可以查询张量所在的设备。默认情况下，张量是在CPU上创建的。

In [37]:
x = tf.constant([1, 2, 3])
x.device

'/job:localhost/replica:0/task:0/device:GPU:0'

需要注意的是，无论何时我们要对多个项进行操作， 它们都必须在同一个设备上。 例如，如果我们对两个张量求和， 我们需要确保两个张量都位于同一个设备上， 否则框架将不知道在哪里存储结果，甚至不知道在哪里执行计算。

